# 3강. Advanced Spark

### key와 value로 데이터를 구분해보자
- Spark는 주어진 Rdd의 row가 가지고 있는 첫번째 column을 key로 인식함
- 대부분의 Key가 필요한 함수들은 접미어로 Key를 가지고 있음
 + ex> sortByKey, reduceByKey

In [1]:
#import findspark; findspark.init()
import pyspark

#sc = pyspark.SparkContext('local[*]', appName="myAppName")

<div class="alert alert-warning"/>
- flatMap(withReplacement, fraction, seed)<br>
 + 주어진 rdd를 순회하면서 복원/비복원 추출을 주어진 비율만큼 주어진 random seed를 사용하여 추출<br><br>
- reduceByKey(withReplacement, fraction, seed)<br>
 + 주어진 rdd를 key를 기준으로 collect후 주어진 func을 수행<br>

In [2]:
#중첩된 형태의 rdd를 하나의 row 단위로 풀어 줄 수 있는 명령, 대단히 중요한 명령
rdd = sc.parallelize([(1, 2), (1, 3), (1, 2), (2, 4)])
print(rdd.flatMap(lambda x:x).collect())
print()

#중첩된 정도에 따라 row로 풀리는 단위가 다름
rdd = sc.parallelize([((1, 2), (1, 3)), ((1, 2), (2, 4))])
print(rdd.flatMap(lambda x:x).collect())

[1, 2, 1, 3, 1, 2, 2, 4]

[(1, 2), (1, 3), (1, 2), (2, 4)]


In [3]:
rdd = sc.parallelize(["a,b", "c,d", "e,f"])
rdd.flatMap(lambda x:x).collect()

['a', ',', 'b', 'c', ',', 'd', 'e', ',', 'f']

In [4]:
rdd.flatMap(lambda x:x.split(",")).collect()

['a', 'b', 'c', 'd', 'e', 'f']

In [5]:
#reduceByKey를 이용하여 key 별로 reduce를 수행할 수 있음
rdd = sc.parallelize([(1, 2), (1, 3), (1, 2), (2, 4), (1, 5)])
rdd.reduceByKey(lambda a,b: a+b).collect()

[(1, 12), (2, 4)]

In [6]:
#groupBy와 유사한 groupByKey동작도 존재함
#reduceByKey는 함수로 처리된 하나의 결과값을 남겨두지만
#groupByKey는 모든 원소를 보유하고 순회할 수 있음
rdd = sc.parallelize([(1, 2), (1, 3), (1, 2), (2, 4), (1, 5)])
grouped = rdd.groupByKey()
print(grouped.collect())

#grouped data중 1이 key인 group 데이터를 추출
a, b = grouped.collect()
print(a[0], list(a[1]))

[(1, <pyspark.resultiterable.ResultIterable object at 0x7fc6fd83a5d0>), (2, <pyspark.resultiterable.ResultIterable object at 0x7fc6fd83a4d0>)]
1 [2, 3, 2, 5]


In [7]:
#key를 group 기준으로 임의로 만들어 준 뒤 groupByKey나 reduceByKey를 돌리는 것이 일반적
#다음 데이터에 짝수, 홀수의 기준으로 key를 분리한뒤 groupByKey로 각 원소를 출력해보자
rdd = sc.parallelize([1,2,3,4,5,6,7,8])
rdd = rdd.map(lambda x:(x % 2 ==0, x))
print ( rdd.groupByKey().collect() )

a, b = rdd.groupByKey().collect()
print (list(a[1]))
print (list(b[1]))

[(False, <pyspark.resultiterable.ResultIterable object at 0x7fc6fe148510>), (True, <pyspark.resultiterable.ResultIterable object at 0x7fc6fe148590>)]
[1, 3, 5, 7]
[2, 4, 6, 8]


<div class="alert alert-warning"/>
- mapValues(func)<br>
 + 주어진 rdd를 순회하면서 key와 value 중 value 부분에 func을 적용<br><br>
- flatMapValues(func)<br>
 + 주어진 rdd를 flat 동작을 value 기준으로 수행하면서 key와 합쳐서 생성해냄<br><br>
- groupByKey(func)<br>
 + rdd의 값을 순회하면서 func가 반환시키는 기준의 데이터로 값들을 묶어주는 동작<br>

In [8]:
#mapValues는 key, value로 나눠진 데이터에 대하여 value 쪽만 function으로 컨트롤 가능
rdd = sc.parallelize(["a", "b", "c"])
rdd = rdd.map(lambda x: (x, 0)).mapValues(lambda x:x+1)
rdd.collect()

[('a', 1), ('b', 1), ('c', 1)]

In [9]:
#flatMapValues를 이용하여 key에 대하여 각 value들을 모두 새로운 기준의 row로 분리 가능
print( rdd.mapValues(lambda x:(x, x)).collect() )
print( rdd.mapValues(lambda x:(x, x)).flatMapValues(lambda x:x).collect() )

[('a', (1, 1)), ('b', (1, 1)), ('c', (1, 1))]
[('a', 1), ('a', 1), ('b', 1), ('b', 1), ('c', 1), ('c', 1)]
